<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/HuggingFace_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141801 sha256=b651456ef3279440159f594d6282d1dcdd25f1aee3096740c4d782ec341fd0d4
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [4]:
!pip install transformers
!pip install sentencepiece

  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1
    Uninstalling tokenizers-0.8.1:
      Successfully uninstalled tokenizers-0.8.1


In [5]:
# Pretokenizer KoNLPy
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

In [29]:
from transformers import *
import tensorflow as tf
from tqdm import tqdm
from typing import Optional, List, Union
import urllib.request
import numpy as np
import pandas as pd
import json
import re
import os

In [7]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [8]:
songys = pd.read_csv("gdrive/My Drive/Colab Notebooks/HelloNewWorld/songysData.csv")

In [9]:
# 결측 row 제거하는 방법, 5232개
songys.dropna(how='any')
songys[:10]

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [10]:
texts = []
pairs = []

for sentence in songys['Q']:
    texts.append(sentence)

for sentence in songys['A']:
    pairs.append(sentence)

list(zip(texts, pairs))[:5]

[('12시 땡!', '하루가 또 가네요.'),
 ('1지망 학교 떨어졌어', '위로해 드립니다.'),
 ('3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('PPL 심하네', '눈살이 찌푸려지죠.')]

In [13]:
class KoNLPyPreTokenizer:
  def __init__(self, konlpy_tagger):
    self.konlpy_tagger = konlpy_tagger
  def __call__(self, sentence):
    return self.pre_tokenize(sentence)
  def pre_tokenize(self, sentence):
    return ' '.join(self.konlpy_tagger.morphs(sentence))

sent_ko = '신종 코로나바이러스 감염증(코로나19) 사태가 심각합니다'
pretok = KoNLPyPreTokenizer(Mecab())
print(f'input : {sent_ko}\npretok : {pretok(sent_ko)}')

input : 신종 코로나바이러스 감염증(코로나19) 사태가 심각합니다
pretok : 신종 코로나 바이러스 감염증 ( 코로나 19 ) 사태 가 심각 합니다


In [14]:
def prepare_pretokenized_corpus(pretokenized_path):
  with open(pretokenized_path, 'w', encoding='utf-8') as f:
    for line in texts:
      f.write(f'{pretok(line)}\n')
    for line in pairs:
      f.write(f'{pretok(line)}\n')

In [15]:
prepare_pretokenized_corpus(
    pretokenized_path = path+"ChatbotData.mecab.txt"
)

In [16]:
from tokenizers import BertWordPieceTokenizer

bert_wordpiece_tokenizer = BertWordPieceTokenizer()

bert_wordpiece_tokenizer.train(
    files = [path+'ChatbotData.mecab.txt'],
    vocab_size = 10000,
    min_frequency = 1,
    limit_alphabet = 1000,
    initial_alphabet = [],
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    show_progress = True,
    wordpieces_prefix = "##",
)
vocab = bert_wordpiece_tokenizer.get_vocab()
print(sorted(vocab, key=lambda x: vocab[x]))

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '!', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', '_', 'a', 'b', 'c', 'd', 'g', 'j', 'k', 'l', 'n', 'o', 'p', 's', 'x', '~', 'ᄀ', 'ᄁ', 'ᄂ', 'ᄃ', 'ᄄ', 'ᄅ', 'ᄆ', 'ᄇ', 'ᄈ', 'ᄉ', 'ᄊ', 'ᄋ', 'ᄌ', 'ᄍ', 'ᄎ', 'ᄏ', 'ᄐ', 'ᄑ', 'ᄒ', 'ᅡ', 'ᅢ', 'ᅣ', 'ᅤ', 'ᅥ', 'ᅦ', 'ᅧ', 'ᅨ', 'ᅩ', 'ᅪ', 'ᅫ', 'ᅬ', 'ᅭ', 'ᅮ', 'ᅯ', 'ᅰ', 'ᅱ', 'ᅲ', 'ᅳ', 'ᅴ', 'ᅵ', 'ᆨ', 'ᆩ', 'ᆪ', 'ᆫ', 'ᆬ', 'ᆭ', 'ᆮ', 'ᆯ', 'ᆰ', 'ᆱ', 'ᆲ', 'ᆶ', 'ᆷ', 'ᆸ', 'ᆹ', 'ᆺ', 'ᆻ', 'ᆼ', 'ᆽ', 'ᆾ', 'ᆿ', 'ᇀ', 'ᇁ', 'ᇂ', '…', 'ㅊ', 'ㅋ', 'ㅎ', 'ㅜ', 'ㅠ', '##ᅡ', '##ᄌ', '##ᅥ', '##ᆼ', '##ᄒ', '##ᅢ', '##ᅳ', '##ᄅ', '##ᅩ', '##ᆨ', '##ᄀ', '##ᆷ', '##ᄃ', '##ᅪ', '##ᆫ', '##ᅵ', '##ᇀ', '##ᄋ', '##ᅮ', '##ᄐ', '##ᄇ', '##ᄎ', '##ᅧ', '##ᆯ', '##ᄆ', '##ᄂ', '##ᆭ', '##ᆻ', '##ᅦ', '##ᄉ', '##ᆺ', '##ᆩ', '##ᅲ', '##ᆸ', '##ᄁ', '##ᅭ', '##ᅱ', '##ᄄ', '##ᅣ', '##ᄑ', '##ᅯ', '##ᄍ', '##ᇁ', '##ㅋ', '##ᆽ', '##ᇂ', '##ᅫ', '##ᆮ', '##ᅬ', '##ᄈ', '##ᆹ', '##ᆾ', '##ᄏ', '##ᅴ', '##ᅨ', '##ᄊ', '##ᆰ', '##ᆿ', '##ᆱ', '##6', '##5', '##0', '##x

In [30]:
bert_wordpiece_tokenizer.save_model(
    directory = path+'tokenizers/MecabBertWordPieceTokenizer/',
    name = 'chtbot'
)

TypeError: ignored

In [26]:
class KoNLPyWordPieceTokenizer:
  def __init(self, konlpy_tagger, wordpieces_prefix='##'):
    self.konlpy_tagger = konlpy_tagger

    def pretokenizer(eojeol):
      # 품사 정보 이용 X
      # return self.konlpy_tagger.morphs(eojeol)
      # 품사 정보 이용 O
      return self.konlpy_tagger.pos(eojeol, join=True)
    self.pretokenize = pretokenize
    self.prefix = wordpieces_prefix

  def tokenize(self, sent):
    def _tokenize(eojeol):
      split_tokens = self.pretokenize(eojeol)
      if len(split_tokens) <= 1:
        return split_tokens
      return [split_tokensp[0]] + [f'{self.prefix}{sub}' for sub in split_tokens[1:]]
    
    split_tokens = []
    for eojeol in sent.split():
      split_tokens += _tokenize(eojeol)
      return split_tokens

  def token_to_alphabets(self, token):
    if token[:2] == self.prefix:
      token = token[2:]
    if self.use_tag:
      return list(token.rsplit('/')[0])
    return list(token)

In [31]:
class KoNLPyBertWordPieceTrainer:
    def __init__(self, konlpy_tagger, wordpieces_prefix="##", use_tag=False):
        konlpy_wordpiece = KoNLPyWordPieceTokenizer(
            konlpy_tagger,
            wordpieces_prefix,
            use_tag
        )
        self.tokenizer = konlpy_wordpiece

    def tokenize(self, sent):
        split_tokens = []
        for eojeol in sent.split():
            split_tokens += self.tokenizer.tokenize(eojeol)
        return split_tokens

    def train(
        self,
        files: Union[str, List[str]],
        vocab_size: int = 30000,
        min_frequency: int = 2,
        limit_alphabet: int = 1000,
        initial_alphabet: List[str] = [],
        special_tokens: List[str] = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
        show_progress: bool = True,
    ):
        if isinstance(files, str):
            files = [files]
        alphabets = initialize_alphabet(files, limit_alphabet, initial_alphabet, special_tokens, show_progress)
        self.vocab = train_vocab(files, vocab_size, min_frequency, show_progress, alphabets, self.tokenizer.tokenize)

    def save_model(self, directory: str, name: Optional[str] = None):
        if name is None:
            name = 'vocab.txt'
        else:
            name = f'{name}-vocab.txt'
        directory = os.path.abspath(directory)
        if not os.path.exists(directory):
            os.makedirs(directory)
        vocab_file = f'{directory}/{name}'
        with open(vocab_file, 'w', encoding='utf-8') as f:
            for subword in self.vocab:
                f.write(f'{subword}\n')
        print(f'[{vocab_file}]')
